## Notebook for Week 3 Assignment - Part 3

#### Step 1 - import pandas library

In [41]:
import pandas as pd

#### Step 2 - scrape the web link using pandas 'read_html' function that returns a list of dataframes (assigned to variable df_list)

In [42]:
df_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

#### Step 3 - our required dataframe is obtained in df_list[0] and assigned to variable df. We check the first 5 rows of df to ensure that our dataframe is read properly

In [43]:
df = df_list[0]

In [44]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Step 4 - remove those entries from df where Borough is not assigned

In [45]:
df = df[df['Borough']!='Not assigned']

#### Step 5 - assignment states that more than one neighborhood can exist in one postal code, and if that is the case, we are expected to combine those rows into one single row with the neighborhoods separated by a comma.

##### However, when we check the data, we see that there are NO repetitions of postal code (number of unique values of postal code = number of rows of data frame as shown below). Therefore, no combining of rows is needed

In [46]:
df['Postal Code'].unique().size == df.shape[0] #number of unique values of Postal Code is same as number of rows in df

True

#### Step 6 - assignment states 'If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough'

##### However, when we check the data, we see that there are NO such neighborhoods that have not been assigned (as shown below)

In [47]:
df[df['Neighborhood']=='Not assigned'].shape[0]

0

#### Step 7 - print the number of rows of the dataframe using shape method

In [48]:
print('The dataframe \'df\' has', df.shape[0], 'rows')

The dataframe 'df' has 103 rows


In [49]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Part 2 starts from here

#### I am using the ArcGIS geocoding service instead of Google because ArcGIS is much more stable and reliable 

In [60]:
import geocoder

#### storing the latitude and longitude information for the postal codes into two lists named - latlist and longlist

In [51]:
latlist = []
longlist = []

for i in range(0,df.shape[0]):
    postcode = df.iloc[i,0]
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postcode))
    lat = g.latlng[0]
    long = g.latlng[1]
    latlist.append(lat)
    longlist.append(long)

#### creating two new columns in our dataframe df and assigning the latlist and longlist to them

In [65]:
df['Latitude'] = latlist
df['Longitude'] = longlist

C:\Users\vshar\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\vshar\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### checking the first 5 rows of dataframe df to ensure that the dataframe is loaded correctly

In [66]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.752935,-79.335641
3,M4A,North York,Victoria Village,43.728102,-79.311890
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


# PART 3 starts here

#### step 1 - extracting only those neighborhoods where the boroughs contains the word 'Toronto' into a new dataframe called 'df2'

In [115]:
boroughlist = ['Downtown Toronto', 'East Toronto', 'West Toronto','Central Toronto']

In [116]:
df2 = pd.DataFrame()

for i in range(0,df.shape[0]):
    temp = df.iloc[i,1]
    if(temp in boroughlist): 
        df2 = df2.append(pd.DataFrame(df.iloc[i,:]).transpose())


##### check how many rows are there in the new dataframe df2

In [118]:
df2.shape[0]

39

#### step 2 - creating a map of the extracted neighborhoods 

In [123]:
import folium # plotting library

# create map of Toronto using latitude and longitude values
g1 = geocoder.arcgis('{}, Toronto, Ontario'.format(postcode))
latitude = g1.latlng[0]
longitude = g1.latlng[1]

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto